In [37]:
PROJECT = "train-to-cloud-city-4"
BASE = "http://localhost:8000/"

In [38]:
import requests
import json
from urllib.parse import urljoin

from google.cloud import firestore
db = firestore.Client(project=PROJECT)


update_time {
  seconds: 1711142482
  nanos: 885573000
}

In [14]:
res = requests.get(urljoin(BASE,"service/"))
# print(res.text)

service_data = json.loads(res.text)
print(service_data["cloud-run"])

with open("../static/category_map.json", encoding="utf-8") as f:
    category_map = json.loads(f.read())

with open("../static/category_list.json", encoding="utf-8") as f:
    category_list = json.loads(f.read())

print(category_map)
#category_map = {key.replace(r"/", r" & "): value for key, value in category_map.items()}
print(category_map)
print(category_list)


{'slug': 'cloud-run', 'name': 'Cloud Run', 'description': 'Serverless for containerized applications', 'doc_url': 'https://cloud.google.com/run/docs'}
{'Compute': ['cloud-functions', 'app-engine', 'cloud-run', 'cloud-run-for-anthos', 'gke', 'compute-engine', 'bare-metal-solution', 'preemptible-vms', 'sole-tenant-nodes', 'vmware-engine'], 'Storage': ['cloud-filestore', 'cloud-storage', 'local-ssd', 'persistent-disk'], 'Database': ['alloydb', 'cloud-bigtable', 'cloud-firestore', 'cloud-memorystore', 'cloud-sql', 'cloud-sql-insights', 'cloud-spanner'], 'Data Analytics': ['bigquery', 'bigquery-bi-engine', 'bigquery-data-transfer-service', 'bigquery-dts', 'bigquery-gis', 'bigquery-ml', 'cloud-composer', 'cloud-data-fusion', 'connected-sheets', 'data-catalog', 'data-studio', 'dataflow', 'dataplex', 'dataprep-by-trifacta', 'dataproc', 'datastream', 'looker', 'public-datasets'], 'AI/ML': ['automl', 'cloud-tpu', 'cloud-translation', 'cloud-vision', 'contact-center-ai', 'deep-learning-containers

In [15]:

services_ref = db.collection("services")

q = services_ref.count()
#q??
print(q.get())

docs = services_ref.stream()

for i, doc in enumerate(docs):
    print(f"{doc.id} => {doc.to_dict()}")
    if i > 5: break

[[<Aggregation alias=field_1, value=248, readtime=2024-03-22 20:06:34.120126+00:00>]]
access-context-manager => {'slug': 'access-context-manager', 'description': 'Fine-grained, attribute based access-control', 'doc_url': 'https://cloud.google.com/access-context-manager/docs', 'name': 'Access Context Manager'}
access-transparency => {'slug': 'access-transparency', 'description': 'Audit cloud provider access', 'doc_url': 'https://cloud.google.com/cloud-provider-access-management/access-transparency/docs/overview', 'name': 'Access Transparency'}
admin-sdk => {'slug': 'admin-sdk', 'description': 'Manage Google Workspace resources', 'doc_url': 'https://developers.google.com/apps-script/advanced/admin-sdk-directory', 'name': 'Admin SDK'}
alloydb => {'slug': 'alloydb', 'description': 'Scalable & performant PostgreSQL - compatible DB', 'doc_url': 'https://cloud.google.com/alloydb/docs', 'name': 'AlloyDB'}
amp-for-email => {'slug': 'amp-for-email', 'description': 'Dynamic interactive email', 'd

In [16]:
# insert/update service data
for key, data in service_data.items():
    doc_ref = services_ref.document(key)
    doc_ref.set(data)

In [17]:

category_ref = db.collection("services_by_category")

q = category_ref.count()
#q??
print(q.get())

docs = category_ref.stream()

for i, doc in enumerate(docs):
    print(f"{doc.id} => {doc.to_dict()}")
    if i > 5: break

[[<Aggregation alias=field_1, value=0, readtime=2024-03-22 20:06:43.619275+00:00>]]


In [35]:
 # map category data

train_services = [
"bigquery", "artifact-registry", "kubernetes", "eventarc", "compute-engine", 
"cloud-functions", "sql", "cloud-build","workflows", "cloud-run", 
"secret-manager", "cloud-firestore", "cloud-storage", "cloud-tasks", 
"cloud-scheduler", "pubsub", "iam", "vpc-network", "cloud-deploy"]

categories = {}
for cat, services in category_map.items():
    cat_slug = cat.lower()
    cat_slug = cat_slug.replace(" ", "-")
    cat_slug = cat_slug.replace("/", "-")
    #print({"services": [service_data[slug] for slug in services]})
    categories[cat_slug] = {"services": [service_data[slug] for slug in services if slug in train_services]}
    categories[cat_slug]["name"] = cat
    #print(cat, services)

print(categories["compute"])
print(len(categories))

to_delete = []
for cat, data in categories.items():
    if len(data["services"]) == 0:
        to_delete.append(cat)
for key in to_delete:
    del(categories[key])
print(len(categories))


{'services': [{'slug': 'cloud-functions', 'name': 'Cloud Functions', 'description': 'Event-driven serverless functions', 'doc_url': 'https://cloud.google.com/functions/docs/'}, {'slug': 'cloud-run', 'name': 'Cloud Run', 'description': 'Serverless for containerized applications', 'doc_url': 'https://cloud.google.com/run/docs'}, {'slug': 'compute-engine', 'name': 'Compute Engine', 'description': 'VMs, GPUs, TPUs, Disks', 'doc_url': 'https://cloud.google.com/compute/docs/'}], 'name': 'Compute'}
21
8


In [36]:
# insert/update service_by_category data
for key, data in categories.items():
    doc_ref = category_ref.document(key)
    #print(doc_ref)
    #print(data)
    doc_ref.set(data)

In [20]:
# add patterns

res = requests.get(urljoin(BASE,"pattern/"))
# print(res.text)
pattern_data = json.loads(res.text)

for pattern in pattern_data.values():
    # print(pattern)
    for checkpoint in pattern["checkpoints"]:
        for service in checkpoint["satisfying_services"]:
            if not ((service in service_data) or (service in {"*"})):
                print (f"'{service}' not found. Referenced in checkpoint: {checkpoint}")

patterns_ref = db.collection("patterns")
for data in pattern_data.values():
    doc_ref = patterns_ref.document(data["slug"])
    doc_ref.set(data)


In [21]:
# TODO: update to new format without 'world' doc

#res = requests.get(urljoin(BASE,"default_world/"))
# print(res.text)
#default_world = json.loads(res.text)
#default_world

#global_ref = db.collection("global")
#global_ref.document("world").set(default_world)


In [22]:
docs = services_ref.stream()
for i, doc in enumerate(docs):
    if i >= 5: break
    print(f"{doc.id} => {doc.to_dict()}")
print("----")
    
docs = patterns_ref.stream()
for i, doc in enumerate(docs):
    if i >= 5: break
    print(f"{doc.id} => {doc.to_dict()}")



access-context-manager => {'slug': 'access-context-manager', 'description': 'Fine-grained, attribute based access-control', 'doc_url': 'https://cloud.google.com/access-context-manager/docs', 'name': 'Access Context Manager'}
access-transparency => {'slug': 'access-transparency', 'description': 'Audit cloud provider access', 'doc_url': 'https://cloud.google.com/cloud-provider-access-management/access-transparency/docs/overview', 'name': 'Access Transparency'}
admin-sdk => {'slug': 'admin-sdk', 'description': 'Manage Google Workspace resources', 'doc_url': 'https://developers.google.com/apps-script/advanced/admin-sdk-directory', 'name': 'Admin SDK'}
alloydb => {'slug': 'alloydb', 'description': 'Scalable & performant PostgreSQL - compatible DB', 'doc_url': 'https://cloud.google.com/alloydb/docs', 'name': 'AlloyDB'}
amp-for-email => {'slug': 'amp-for-email', 'description': 'Dynamic interactive email', 'doc_url': 'https://developers.google.com/apps-script/add-ons/gmail', 'name': 'AMP for E